# CREATING A BOILERPLATE DJANGO PROJECT WITH CONFIG

### Startproject

Creates a Django project directory 


` django-admin startproject <project_name>`

### Start app 

1. Create a users app (if necessary)

If the application is going to be dealig with user log ins, the first thing I do is create a `users` app in the file directory. 

Why?  
Allows you to override the initial users config.  
Later, will change settings.py to add the `AUTH_USER_MODEL`

`python manage.py startapp <appname>`

### Change the directory name

By default the folder which holds `settings.py , urls.py, wsgi.py` is named after the prject itself. 

It is my preference to change the name to `config`.  
  
Then: 
- Go to `asgi.py, settings.py, wsgi.py` and change `os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'config.settings')`  

- and in `settings.py` change `ROOT_URLCONF = 'config.urls'`
- `WSGI_APPLICATION `

### settings.py

Go to `settings.py`  
  
Add:  
- `import os`  
- `from decouple import config` - i use decouple 
  
  
Create mapping to directories  
Below `BASE_DIR`  
- `TEMPLATE_DIR = os.path.join(BASE_DIR, 'templates')` - TEMPLATES
- `STATIC_DIR = os.path.join(BASE_DIR, "static")` - STATIC  
  
  
create a `.env` file - for environment variables.   

- SECRET_KEY  
- DEBUG  
Add any other variables to the `.env` file
  
  
Setup Static url and staticfiles_dirs  

```python

STATIC_URL= config('STATIC_URL')
STATICFILES_DIRS= [
    STATIC_DIR,
]
```

Add redirect urls  

```python
LOGIN_REDIRECT_URL = ''  
LOGOUT_REDIRECT_URL = ''
``` 
Fill later  

Setup media directories  
  
`MEDIA_ROOT = os.path.join(BASE_DIR, `  `'media') # Directory where uploaded media is saved.`  
`MEDIA_URL = '/media/' # Public URL at the browser`  


### Add PostgreSQL settings  

```python
    DATABASES = {
        'default': {
            'ENGINE': 'django.db.backends.postgresql_psycopg2',
            'NAME': 'usersDB', 
            'USER': 'sammy', 
            'PASSWORD': 'pa$$word',
            'HOST': '127.0.0.1', 
            'PORT': '5432',
        }
    }
```
  
Then add details to `.env` file

### Setup users & auth
  
Go to `users` app created earlier  
  
##### Paste boilerplate code into `models.py` 
  
```python

from django.contrib.auth.models import AbstractUser
from django.db import models
from django.db import models
from django_countries.fields import CountryField
from django.contrib.auth.models import (
    BaseUserManager, AbstractBaseUser)


class CustomUser(AbstractUser):
    pass

class CustomUserManager(BaseUserManager):
    def create_user(self, email, password=None):
        if not email:
            raise ValueError("users must have an email address")

        user = self.model(
            email=self.normalize_email(email),
        )

        user.set_password(password)
        user.is_admin = False
        user.save(using=self.db)
        return user

    def create_superuser(self, email, password=None):
        user = self.create_user(
            email,
            password=password,
        )
        user.is_admin = True
        user.is_staff = True
        user.save(using=self.db)
        return user

class User(AbstractBaseUser):
    email = models.EmailField(
        verbose_name='email address', 
        max_length=255,
        unique=True,
    )
    is_student = models.BooleanField(default=False) 
    is_active = models.BooleanField(default=True)
    is_admin = models.BooleanField(default =False)
    is_staff = models.BooleanField(default=True)


    USERNAME_FIELD='email'

    objects = CustomUserManager()

    def __str__(self):
        return self.email

    def has_perm(self,perm,obj=None):
        "Does the user have a specific permission"
        return True

    def has_module_perms(self,app_label):
        "Does the user have permissions to view the app 'app_label'?"
        return True

class UserProfile(models.Model):
    user = models.OneToOneField(
        CustomUser,
        on_delete=models.CASCADE,
        primary_key=True,
    )
    # detail
    first_name = models.CharField(max_length=100)
    last_name = models.CharField(max_length=100)
    preferred_name = models.CharField(max_length=100)
    current_job_title = models.CharField(max_length=255)
    website = models.CharField(max_length=255)

    # log ins
    discord_name = models.CharField(max_length=100)
    image = models.ImageField(upload_to="media/profile-images")
    github_username = models.CharField(max_length = 100)
    codepen_username = models.CharField(max_length=100)
    fcc_profile_url = models.CharField(max_length=255)
    LEVELS = (
        (1, 'Level One'),
        (2, 'Level Two'),
    )
    current_level = models.IntegerField(choices=LEVELS)
    phone = models.CharField(max_length=50)

    #  Location
    timezone = models.CharField(max_length=50)
    country = CountryField(multiple=False),
    street_address = models.CharField(max_length=100)
    apartment_address = models.CharField(max_length=100)
    zip = models.CharField(max_length=100)


    def __str__(self):
        return f'{self.first_name} {self.last_name}'


```
  

##### In forms.py paste boilerplate code  
  
```python
# users/forms.py
from django.contrib.auth import get_user_model
from django.contrib.auth.forms import UserCreationForm, UserChangeForm


class CustomUserCreationForm(UserCreationForm):

    class Meta:
        model = get_user_model()
        fields = ('email', 'username',)



class CustomUserChangeForm(UserChangeForm):
    class Meta:
        model = get_user_model()
        fields = ('email', 'username',)



```

##### In settings.py  
  

1. add `users` to installed apps `'users.apps.UsersConfig',`
2. add `AUTH_USER_MODEL = 'users.CustomUser'`


### Setup urls.py
  

```python 
from django.contrib import admin
from django.urls import path, include
from django.conf import settings
from django.conf.urls.static import static
from django.contrib.auth import get_user_model

urlpatterns = [
    # Admin
    path('admin/', admin.site.urls),

    # User management
    path('accounts/', include('django.contrib.auth.urls')),

]

urlpatterns += static(settings.MEDIA_URL, document_root=settings.MEDIA_ROOT)
```



### Migrations

`python manage.py makemigrations`  
  
`python manage.py migrate`